<a href="https://colab.research.google.com/github/thuhinkhanna/COMP-6721-Project/blob/main/AI_Project_Fashion_Products_Images_Small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paramaggarwal/fashion-product-images-small
!unzip fashion-product-images-small.zip

In [ ]:
import pandas as pd
df = pd.read_csv('/content/myntradataset/styles.csv', error_bad_lines=False)

In [ ]:
df.shape

The dataset contains 44424 rows and 10 classes.

In [ ]:
df.head(10)

In [ ]:
df['masterCategory'].unique()

In [ ]:
category_counts = df['masterCategory'].value_counts()
print(category_counts)

In [ ]:
import matplotlib.pyplot as plt
category_counts.plot(kind='bar', color='skyblue', figsize=(10, 6))
plt.xlabel('Categories')
plt.ylabel('Counts')
plt.title('Count of Items per Category')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability if needed
plt.tight_layout()
plt.show()

From the graph it can be seen that the three master categories Free items, Sporting Goods and Home datas are negligible.


In [ ]:
!ls /content/images | wc -l

The number of images are different from the number of Ids present.

In [ ]:
import os,numpy as np
image_folder = 'myntradataset/images'
image_files = os.listdir(image_folder)
dataset_ids = df['id'].astype(str) + '.jpg'
dataset_ids_set = set(dataset_ids)
image_files_set = set(image_files)
#Missing images are calculated.
missing_images = dataset_ids_set - image_files_set
#Extra images with no corresponding ids are calculated.
extra_images = image_files_set - dataset_ids_set
print(missing_images)
print(extra_images)

print(df['id'].dtype)

In [ ]:

missing_imgs = list(missing_images)
missing=[]
for i in missing_imgs:
  missing.append(np.int64(i[:-4]))
df=df[~df['id'].isin(missing)]
df = df.loc[~df['id'].isin(missing), :]
print(df.shape)
# New column named 'image' is created with no id column.
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df.head()


In [ ]:
import os

base_dir = '/content/Images_'

# Create the directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Create subfolders(List of classes).
subfolders = ['Apparel', 'Accessories', 'Footwear', 'Personal Care']
for folder in subfolders:
    folder_path = os.path.join(base_dir, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
import os
import shutil
from PIL import Image

# Function to resize an image
def resize_image(image_path, output_size):
    img = Image.open(image_path)
    img_resized = img.resize(output_size, Image.ANTIALIAS)
    return img_resized

category_mapping = {
    'Accessories': '/content/Images_/Accessories',
    'Apparel': '/content/Images_/Apparel',
    'Footwear': '/content/Images_/Footwear',
    'Personal Care': '/content/Images_/Personal Care',
}

# Set to keep track of moved items
moved_items = set()

# Output size for resized images (e.g., (width, height))
output_size = (224, 224)  # Adjust as needed

for index, row in df.iterrows():
    item_id = row['id']
    master_category = row['masterCategory']
    item = str(item_id) + '.jpg'

    if master_category in category_mapping:
        source_path = '/content/myntradataset/images/' + item
        destination_path = category_mapping[master_category]

        # Check if the item has already been moved
        if item not in moved_items:
            # Check if destination file already exists
            if not os.path.exists(os.path.join(destination_path, item)):
                # Resize image
                resized_image = resize_image(source_path, output_size)
                # Save resized image to destination
                resized_image.save(os.path.join(destination_path, item))
                # Add item to set of moved items
                moved_items.add(item)
            else:
                print(f"Destination file {item} already exists in {destination_path}. Skipping...")
        else:
            print(f"Item {item} already moved. Skipping...")


In [ ]:
#Checking if the images have been resized to 224*224 to train the model.
from PIL import Image
img = Image.open("/content/Images_/Accessories/10734.jpg")

# Get the dimensions of the image
width, height = img.size

# Print the dimensions
print("Width:", width)
print("Height:", height)

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

In [ ]:
#Function to split data into train, test and validation data and perform normalization.
def load_data(path, test_split, val_split, batch_size):
    transform_dict = {
    'src': transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5]),
     ])}

    data = datasets.ImageFolder(root=path, transform=transform_dict['src'])
    dataset_size = len(data)
    test_size = int(test_split * dataset_size)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - (test_size + val_size)

    train_dataset, test_dataset, val_dataset = td.random_split(data,
                                               [train_size, test_size, val_size])

    data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=0)
    data_loader_test  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=0)
    data_loader_val   = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=0)
    return data_loader_train, data_loader_test, data_loader_val

In [ ]:
path='/content/Images_'
train_loader, test_loader, val_loader = load_data(path, 0.3, 0.1, 100)

In [ ]:
import torch
import torchvision.models as models
import torch.optim as optim

# Loading the ResNet-18 model
model = models.resnet18(pretrained=False)

# Modify the last layer for 4 classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 4)

# Define optimizer and criterion
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Moving the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Device: {}".format(device))


In [ ]:
import torch
import numpy as np

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 5

for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):

        inputs, labels = inputs.cuda().to(device), labels.cuda().to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Train Loss: {loss.item():.4f}')

    train_loss /= len(train_loader.dataset)
    train_accuracy = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(val_loader):

            inputs, labels = inputs.cuda().to(device), labels.cuda().to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(val_loader)}], Val Loss: {loss.item():.4f}')

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}')

print('Training complete')

In [ ]:
#Evaluation to check the test accuracy.
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, (correct / total) * 100))

In [ ]:
import matplotlib.pyplot as plt

# training and validation loss

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, 11), train_losses, label='Training Loss')
plt.plot(range(1, 11), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plotting training and validation accuracy

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim



model = models.vgg16(pretrained=False)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
)
for param in model.classifier[6].parameters():
    param.requires_grad = True
    # Define optimizer and criterion
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
path='/content/Images_'
train_loader, test_loader, val_loader = load_data(path, 0.3, 0.1, 100)

In [ ]:
import torch
import numpy as np

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 10

for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):

        inputs, labels = inputs.cuda().to(device), labels.cuda().to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Train Loss: {loss.item():.4f}')

    train_loss /= len(train_loader.dataset)
    train_accuracy = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(val_loader):

            inputs, labels = inputs.cuda().to(device), labels.cuda().to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(val_loader)}], Val Loss: {loss.item():.4f}')

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}')

print('Training complete')

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, (correct / total) * 100))

In [ ]:
import matplotlib.pyplot as plt

# training and validation loss

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1,  num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plotting training and validation accuracy

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()